In [ ]:
import os
import json
import asyncio
from pathlib import Path
from groq import AsyncGroq
from dotenv import load_dotenv

FILE_PROMPT = "prompt.txt"
HISTORY_FILE = 'context_window.json'

async def load_context_window():
    context_path = Path(HISTORY_FILE)
    if context_path.exists():
        try:
            with open (context_path, 'r') as f:
                return json.load(f)
        except (json.JSONDecodeError, IOError):
            print("Warning: Json File is Empty")
            return [{"role": "system", "content": "Halo"}]
        else:
            print("Eror: Cannot load chat history file")
            exit(1)


history_string = await load_context_window()


async def load_system_prompt(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: File prompt tidak ditemukan di {file_path}")
        return None
    


client = AsyncGroq(
    api_key=""
)


prompt = await load_system_prompt(FILE_PROMPT)


async def chunks(prompt) -> None:
    try:
        # Kita ganti nama variabel agar lebih jelas
        history_data = history_string # Ini adalah list dari line 17
        history_as_string = json.dumps(history_data, indent=2) 
    except TypeError as e:
        print(f"Error mengubah 'history' menjadi string JSON: {e}")
        return
    
    response = await client.chat.completions.create(
        messages = [
            {
                "role": "system",
                "content": prompt,
            },
            {
                "role": "user",
                "content": history_as_string,
            }
        ],
        temperature = 0.1,
        model= "llama-3.1-8b-instant",
    )
    output_ril = response.choices[0].message.content
    try:
        data = json.loads(output_ril)


        with open("chat_chunks.json", 'w') as file:
            json.dump(data, file, indent=2)
    except json.JSONDecodeError:
        print("Error: AI tidak mengembalikan JSON yang valid. Menyimpan sebagai file teks.")
        # Fallback: Simpan sebagai file .txt jika AI gagal
        with open("chat_chunks_error.txt", 'w') as file:
            file.write(output_ril)

await chunks(prompt)